In [1]:
import names
import random

In [2]:
class DateGen:
    def __init__(self, year_range):
        self.year_range = year_range
    def gen_full(self):
        month = self.__gen_month()
        year = self.__gen_year()
        day = -1
        if (year % 4 == 0):
            year -= 1
        if (month == 2):
            day = random.randint(1, 29)
        else:
            day = random.randint(1, 30)
        if (month < 10):
            if (day < 10):
                return f'{year}-0{month}-0{day}'
            return f'{year}-0{month}-{day}'
        if (day < 10):
                return f'{year}-{month}-0{day}'
        return f'{year}-{month}-{day}'
    
    def __gen_month(self):
        return random.randint(1, 12)
    def __gen_year(self):
        return random.randint(self.year_range[0], self.year_range[1])

In [3]:
class StandartGenerator:
    def __init__(self):
        self.countries = ['UK', 'Italy', 'Russia', 'Ukraine', 'Scotland']
        self.id = 0
    def gen_id(self):
        self.id += 1
        return self.id - 1
    def gen_fname(self):
        return names.get_first_name()
    def gen_sname(self):
        return names.get_last_name()
    def gen_country(self):
        return self.countries[random.randint(0, len(self.countries)-1)]
        
        

In [4]:
class Inserter:
    def __init__(self, table_characteristic):
        self.table_name = table_characteristic[0]
        self.col_names = table_characteristic[1]
    def make_request(self, params : list):
        print('insert', end = " ")
        print(f'into {self.table_name}')
        print(str('     (') + ', '.join(self.col_names) + str(')'))
        print('values', end = " ")
        print(f'({str(params)[1:-1]});')

In [5]:
class Table:
    def __init__(self, table_characteristic):
        self.inserter = Inserter(table_characteristic)
        self.count = 0
        self.items = list()
    def insert(self, new_player):
        self.items.insert(len(self.items), new_player)
    def clean(self):
        self.items = list()
    def make_requests(self):
        for item in self.items:
            self.inserter.make_request(item)
            print()
        
player_table = ('player', ('id', 'country', 'firstname', 'surname', 'birthdate', 'begindate'))
players = Table(player_table)
coach_table = ('coach', ('id', 'country', 'firstname', 'surname', 'birthdate'))
coaches = Table(coach_table)

In [6]:
class PlayerBuilder:
    def __init__(self):
        self.birth_date_gen = DateGen([1980, 2000])
        self.begin_date_gen = DateGen([2010, 2023])
        self.standart_gen = StandartGenerator()
    def gen_player(self):
        player = list()
        player.insert(len(player), self.standart_gen.gen_id())
        player.insert(len(player), self.standart_gen.gen_country())
        player.insert(len(player), self.standart_gen.gen_fname())
        player.insert(len(player), self.standart_gen.gen_sname())
        player.insert(len(player), self.birth_date_gen.gen_full())
        player.insert(len(player), self.begin_date_gen.gen_full())
        players.insert(player)
class CoachBuilder:
    def __init__(self):
        self.birth_date_gen = DateGen([1970, 1990])
        self.standart_gen = StandartGenerator()
    def gen_coach(self):
        coach = list()
        coach.insert(len(coach), self.standart_gen.gen_id())
        coach.insert(len(coach), self.standart_gen.gen_country())
        coach.insert(len(coach), self.standart_gen.gen_fname())
        coach.insert(len(coach), self.standart_gen.gen_sname())
        coach.insert(len(coach), self.birth_date_gen.gen_full())
        coaches.insert(coach)
class TournamentBuilder:
    def __init__(self):
        

In [55]:
tournament_table = ('tournament', ('id', 'name', 'sponsor', 'elo_rate', 'date', 'country'))
tournaments = Table(tournament_table)
class TourBuilder:
    def __init__(self):
        self.date_gen = DateGen([2012, 2023])
        self.standart_gen = StandartGenerator()
        self.sponsors = [self.standart_gen.gen_fname() +' '+ self.standart_gen.gen_sname() for i in range(5)]
    def gen_tournament(self):
        tournament = list()
        tournament.insert(len(tournament), self.standart_gen.gen_id())
        tournament.insert(len(tournament), str('tournament_' + str(tournament[0])))
        tournament.insert(len(tournament), self.sponsors[random.randint(0, len(self.sponsors)-1)])
        tournament.insert(len(tournament), random.randint(2251, 2825))
        tournament.insert(len(tournament), self.date_gen.gen_full())
        tournament.insert(len(tournament), self.standart_gen.gen_country())
        tournaments.insert(tournament)

In [248]:
match_table = ('match', ('id', 'black_player_id', 'white_player_id', 'duration', 'winner', 'tournament_id'))
matches = Table(match_table)

def day_cnt(date : str):
    return int(date[0:4])*365 + int(date[5:7])*30 + int(date[9:11])

def to_lim(lst : list, max_size = 1000000):
    while len(lst) > max_size:
        lst.pop()

class MatchBuilder:
    def __init__(self):
        self.standart_gen = StandartGenerator()
    def gen_Matches(self, tournament_id):
        self.tournament_id = tournament_id
        date = tournaments.items[tournament_id][4]
        players_ = [player for player in players.items if (day_cnt(player[5]) >= day_cnt(date))]
        to_lim(players_, 1000)
        print(len(players_))
        self.make_tournament_grid(players_)
    def make_tournament_grid(self, use_players : list):
        winners = list()
        loozers = list()
        for i in range(int(len(use_players)/2)):
            item = list()
            match = self.make_match(use_players[i], use_players[int(len(use_players) / 2) + i])
            winners.append(match['winner'])
            if (use_players[i] == match['winner']):
                loozers.append(use_players[int(len(use_players) / 2) + i])
            else:
                loozers.append(use_players[i])
            
            item.append(self.standart_gen.gen_id())
            item.append(match['black'][0])
            item.append(match['white'][0])
            item.append(random.randint(20, 80))
            if (match['black'] == match['winner']):
                item.append(1)
            else:
                item.append(0)
            item.append(self.tournament_id)    
            
            matches.insert(item)
            
        if (len(use_players) > 3):
            self.make_tournament_grid(winners)
            self.make_tournament_grid(loozers)
            
    def make_match(self, player1, player2):
        if (random.randint(0, 1) == 1):
            if (random.randint(0, 1) == 1):
                return {'black':player1, 'white':player2, 'winner':player1}
            else:
                return {'black':player1, 'white':player2, 'winner':player2}
        else:
            if (random.randint(0, 1) == 1):
                return {'black':player2, 'white':player1, 'winner':player1}
            else:
                return {'black':player2, 'white':player1, 'winner':player2}

In [249]:
coaching_table = ('coaching', ('coach_id', 'player_id', 'valid_from', 'valid_to'))
coachings = Table(coaching_table)
class CoachingBuilder:
    def __init__(self):
        self.standart_gen = StandartGenerator()
    def gen_coaching(self):
        for i in range(len(players.items)):
            item = list()
            item.append(coaches.items[i % len(coaches.items)][0])
            item.append(players.items[i][0])
            item.append(players.items[i][5])
            item.append('5999-01-01')
            coachings.insert(item)
            

In [250]:
match_builder = MatchBuilder()
for i in range(len(tournaments.items)):
    match_builder.gen_Matches(i)

35
82
28
25
20
99
81
109
73
73


In [ ]:

with black as (
select 
	player.id as id
	, count(country) as black_wins
from
	player join match
	on player.id = match.black_player_id
group by player.id
),
white as (
select 
	player.id as id
	, count(country) as white_wins
from
	player join match
	on player.id = match.white_player_id
where winner = 0
group by player.id
)
select
	id
	, firstname
	, surname
	, case 
		when white_wins is null and black_wins is null then 0
		when white_wins is null then black_wins
		when black_wins is null then white_wins
		else white_wins + black_wins
		end as wins
from
	white 
	full join black
	using(id)
	full join (select id, firstname, surname from player) as all_
	using(id)
order by wins desc


In [251]:
matches.make_requests()

insert into match
     (id, black_player_id, white_player_id, duration, winner, tournament_id)
values (0, 3, 50, 56, 1, 0);

insert into match
     (id, black_player_id, white_player_id, duration, winner, tournament_id)
values (1, 59, 6, 57, 0, 0);

insert into match
     (id, black_player_id, white_player_id, duration, winner, tournament_id)
values (2, 7, 65, 30, 0, 0);

insert into match
     (id, black_player_id, white_player_id, duration, winner, tournament_id)
values (3, 66, 10, 42, 0, 0);

insert into match
     (id, black_player_id, white_player_id, duration, winner, tournament_id)
values (4, 13, 75, 36, 1, 0);

insert into match
     (id, black_player_id, white_player_id, duration, winner, tournament_id)
values (5, 15, 76, 44, 1, 0);

insert into match
     (id, black_player_id, white_player_id, duration, winner, tournament_id)
values (6, 16, 84, 69, 1, 0);

insert into match
     (id, black_player_id, white_player_id, duration, winner, tournament_id)
values (7, 20, 85, 24, 0, 

insert into coaching
     (coach_id, player_id, valid_from, valid_to)
values (0, 0, '2015-04-09', '5999-01-01');

insert into coaching
     (coach_id, player_id, valid_from, valid_to)
values (1, 1, '2017-03-22', '5999-01-01');

insert into coaching
     (coach_id, player_id, valid_from, valid_to)
values (2, 2, '2019-01-13', '5999-01-01');

insert into coaching
     (coach_id, player_id, valid_from, valid_to)
values (3, 3, '2023-08-10', '5999-01-01');

insert into coaching
     (coach_id, player_id, valid_from, valid_to)
values (4, 4, '2018-09-23', '5999-01-01');

insert into coaching
     (coach_id, player_id, valid_from, valid_to)
values (5, 5, '2019-03-17', '5999-01-01');

insert into coaching
     (coach_id, player_id, valid_from, valid_to)
values (6, 6, '2021-11-18', '5999-01-01');

insert into coaching
     (coach_id, player_id, valid_from, valid_to)
values (7, 7, '2021-10-04', '5999-01-01');

insert into coaching
     (coach_id, player_id, valid_from, valid_to)
values (8, 8, '201